This notebook takes in the raw probabilities file (from `content/posts/2023/2023-12-17-probability-estimates-2024`) and:
1. Transforms it so that each person gets a row, and each event is a column
2. Adds a row of "average" predictions at the bottom
3. Adds a set of *resolutions*, answers as to whether each predicted event actually occurred.
4. Calculates Brier scores for each person as well as for the average predictions.

In [1]:
import polars as pl

In [2]:
raw = pl.read_csv('raw_probabilities.csv')

In [63]:
raw

Name,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""Raluca Pavel""",0.26,0.15,0.0,1.0,1.0,0.9,1.0,0.8,1.0,0.48,0.52,0.0,0.0,1.0,0.2,0.3,0.5,0.65,0.0,1.0,0.0,0.7,0.1,0.0,0.2,0.1,0.87,1.0,0.95,0.68,1.0,0.8,0.95,0.45,0.45,0.45,1.0,1.0,1.0,0.

In [4]:
raw = raw.drop(['Timestamp', "Can I track your answers on my website alongside others'?"])
raw = raw.rename({"What's your name?": "Name"})

In [118]:
no_resolutions = [
    'US Politics | Joe Biden is re-elected president.',
    "US Politics | A Republican candidate who isn't Donald Trump is elected president."
]
presumed_no_resolutions = [
    'Geopolitics | China lands ground forces on the island of Taiwan',
    'Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year',
    'Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.',
    "US Politics | A Democrat candidate who isn't Joe Biden is elected president.",
    'US Politics | 2024 elections give Democrats control of the House of Representatives.',
    'US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.',
    'US Politics | The US president elect dies between the election and the end of 2024.',
    'US Politics | 2024 elections give Democrats control of the Senate.',
    'US Politics | At least one member of the US Supreme Court steps down or dies during 2024.',
    'US Politics | Donald Trump is convicted of a federal crime.',
    'US Politics | Joe Biden is impeached.',
    'Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.',
    'Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.',
    'Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.',
    'Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).',
    'Business & Tech | OpenAI releases (or allows a public preview of) a product branded ""GPT-5"" or ""ChatGPT-5"".',
    'Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.',
    'Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.',
    'Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.',
]

yes_resolutions = [
    'Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.',
    'Sports & Culture | LeBron James and his son LeBron Jr (aka ""Bronny"") are under contract with the same NBA team at any point during 2024.',
    'Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.',
    # thank goodness this is true whether it refers to men's or women's basketball
    'Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.',
    'Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.',
]
presumed_yes_resolutions = [
    'Geopolitics | Xi Jinping is still the premier of China at the end of 2024.',
    'Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.',
    'Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.',
    'Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.',
    'Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.',
    'Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.',
    'US Politics | Donald Trump is elected president.',
    'US Politics | President Jimmy Carter is still alive at the end of 2024.',
    'Business & Tech | The S&P 500 closes the year higher than it began.',
    'Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.',
    'Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.',
    'Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.',
    'Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.',
    'Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.',   
]

# Temporarily, we have "presumptions" -- we'll get rid of them by the end of the year.
# In order to do calculations in the meantime, we'll just take our presumptions as accurate.
yes_resolutions = yes_resolutions + presumed_yes_resolutions
no_resolutions = no_resolutions + presumed_no_resolutions

In [119]:
# Make sure we included every question.
assert set(raw.columns) == set(yes_resolutions + no_resolutions + ['Name'])

In [120]:
props = [col for col in raw.columns if col != 'Name']
resolutions = pl.DataFrame({prop: float(prop in yes_resolutions) for prop in props})
resolutions

Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0


In [121]:
averages = raw.mean()
averages = averages.with_columns(pl.lit('Average').alias("Name"))

estimates = raw.vstack(averages)
estimates.tail()

Name,Geopolitics | China lands ground forces on the island of Taiwan,Geopolitics | Russia and Ukraine sign a truce to stop all combat operations and honor it until the end of the year,Geopolitics | Nuclear weapons are used in an attack (not just tested) somewhere in the world.,Geopolitics | Xi Jinping is still the premier of China at the end of 2024.,Geopolitics | Narendra Modi is still the prime minister of India at the end of 2024.,Geopolitics | Benjamin Netanyahu is still the prime minister of Israel at the end of 2024.,Geopolitics | Vladimir Putin is still the president of Russia at the end of 2024.,Geopolitics | Vlodymyr Zelenskyy is still the president of Ukraine at the end of 2024.,Geopolitics | Pope Francis (Jorge Mario Bergoglio) is still the active pope at the end of 2024.,US Politics | Donald Trump is elected president.,US Politics | Joe Biden is re-elected president.,US Politics | A Republican candidate who isn't Donald Trump is elected president.,US Politics | A Democrat candidate who isn't Joe Biden is elected president.,US Politics | At least one third-party US presidential candidate gets 3% or more of the popular vote.,US Politics | The US president elect dies between the election and the end of 2024.,US Politics | 2024 elections give Democrats control of the Senate.,US Politics | 2024 elections give Democrats control of the House of Representatives.,US Politics | At least one member of the US Supreme Court steps down or dies during 2024.,US Politics | President Jimmy Carter is still alive at the end of 2024.,US Politics | Donald Trump is convicted of a federal crime.,US Politics | Joe Biden is impeached.,Sports & Culture | Mainstream news outlets report definitively that Taylor Swift and Travis Kelce have broken up.,Sports & Culture | Taylor Swift releases an album of composed primarily of entirely new music.,"Sports & Culture | LeBron James retires from or otherwise leaves the NBA, and does not change his mind before the end of the year.","Sports & Culture | LeBron James and his son LeBron Jr (aka """"Bronny"""") are under contract with the same NBA team at any point during 2024.","Sports & Culture | Lionel Messi retires from or otherwise leaves the MLS, and does not change his mind before the end of the year.",Sports & Culture | The US wins the most gold medals of any country (or ties for first) in the Paris Olympics.,Sports & Culture | The US wins the gold medal in basketball at the Paris Olympics.,"Sports & Culture | One of the three championship favorites (Boston Celtics, Denver Nuggets, Milwaukee Bucks) wins the 2024 NBA title.",Business & Tech | The S&P 500 closes the year higher than it began.,Business & Tech | The Fed raises interest rates at least once in 2024 (even if they also decrease rates).,Business & Tech | Sam Altman is still the CEO of OpenAI at the end of 2024.,"Business & Tech | OpenAI releases (or allows a public preview of) a product branded """"GPT-5"""" or """"ChatGPT-5"""".","Business & Tech | Meta is ordered to divest a business unit, by federal court or commission.","Business & Tech | Amazon is ordered to divest a business unit, by federal court or commission.","Business & Tech | Alphabet is ordered to divest a business unit, by federal court or commission.","Business & Tech | The X/Twitter platform still exists at the end of the year: you can log in, share content, and view content.",Business & Tech | Nvidia remains in the top 10 American companies by market cap at the end of 2024.,Business & Tech | Tim Cook is still the CEO of Apple at the end of the year.,Business & Tech | Andy Jassy is still the CEO of Amazon at the end of the year.
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""David Schmitz""",0.03,0.2,0.001,0.995,0.9,0.9,0.97,0.95,0.94,0.4,0.3,0.1,0.2,0.07,0.06,0.6,0.3,0.1,0.02,0.5,0.03,0.4,0.7,0.6,0.09,0.2,0.4,0.8,0.5,0.6,0.5,0.7,0.7,0.05,0.06,0.08,0.96,0.4,

In [122]:
from dataclasses import dataclass
from typing import Self

@dataclass
class Event:
    id: str
    category: str
    proposition: str

    @classmethod
    def from_id(cls, id: str) -> Self:
        category, proposition = id.split(' | ')
        return cls(id=id, category=category, proposition=proposition)

In [123]:
event_ids = [col for col in estimates.columns if col != 'Name']
events = [Event.from_id(event_id) for event_id in event_ids]
categories = {event.category for event in events}
categories

{'Business & Tech', 'Geopolitics', 'Sports & Culture', 'US Politics'}

Compute Brier scores, both for each category and overall.

In [124]:
def brier_score(predictions: tuple[float, ...], resolutions: tuple[float, ...]) -> float:
    """
    Given a list of predictions (floats from 0 to 1) and a same-length list of
    resolutions (0s and 1s), calculate the Brier score.
    """
    scores = [
        (pred - res) ** 2
        for pred, res in zip(predictions, resolutions)
    ]
    return sum(scores) / len(scores)

In [125]:
@dataclass
class PersonScore:
    name: str
    category_scores: dict[str, float]
    overall_score: float

    def to_dict(self) -> dict[str, str | float]:
        d = dict(name=self.name, overall_score=self.overall_score)
        for cat in self.category_scores:
            d[cat] = self.category_scores[cat]
        return d

In [126]:
def scores_for_person(name: str) -> PersonScore:
    category_scores = {}
    predictions = estimates.filter(pl.col('Name') == name)[props]
    overall = brier_score(predictions.row(0), resolutions[props].row(0))
    for category in categories:
        category_props = [prop for prop in props if prop.startswith(category)]
        cat_predictions = predictions[category_props].row(0)
        cat_resolutions = resolutions[category_props].row(0)
        score = brier_score(cat_predictions, cat_resolutions)
        category_scores[category] = score
    return PersonScore(
        name=name,
        category_scores=category_scores,
        overall_score=overall,
    )

people = estimates['Name']
scores = [scores_for_person(person) for person in people]

In [127]:
data = pl.DataFrame(score.to_dict() for score in scores)
data

name,overall_score,Sports & Culture,Geopolitics,US Politics,Business & Tech
str,f64,f64,f64,f64,f64
"""Raluca Pavel""",0.2280525,0.246175,0.015567,0.361942,0.242664
"""Ethan Swan""",0.132155,0.19375,0.070978,0.168467,0.0978
"""Tommy Callan""",0.23325,0.2890625,0.072222,0.479583,0.055682
"""Greg Moore""",0.144198,0.216825,0.008622,0.210733,0.129718
"""Abhisek Gupta""",0.172653,0.235025,0.135011,0.22735,0.098418
…,…,…,…,…,…
"""David Schmitz""",0.145998,0.2660125,0.007547,0.18165,0.1331
"""Phil Anderson""",0.13699,0.2890875,0.021433,0.2069,0.044655
"""Brian Van Fossen""",0.128275,0.246875,0.038344,0.170283,0.069773


In [128]:
import altair as alt

alt.Chart(data).mark_bar().encode(
    x=alt.X('name', sort='y'),
    y='overall_score',
    color=alt.condition(
        alt.datum.name == 'Average',
        alt.value('orange'),
        alt.value('blue'),
    ),
).properties(title='Overall Score')

alt.Chart(...)

In [129]:
cat_charts = []
for category in categories:
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('name', sort='y'),
        y=category,
        color=alt.condition(
            alt.datum.name == 'Average',
            alt.value('orange'),
            alt.value('blue'),
        ),
    ).properties(title=category)
    cat_charts.append(chart)

from functools import reduce
reduce(lambda a, b: a | b, cat_charts)

alt.HConcatChart(...)

Score contributions

In [131]:
def scorecontrib_for_person(name: str) -> dict[str, float]:
    category_scores = {}
    predictions = estimates.filter(pl.col('Name') == name)[props]
    prop_scores = [
        (prop, (pred - res) ** 2)
        for (prop, pred, res) in zip(props, predictions.row(0), resolutions[props].row(0))
    ]
    return sorted(prop_scores, key=lambda t: t[1])

charts = []
for person in estimates['Name']:
    contrib = pl.DataFrame(
        scorecontrib_for_person(person),
        orient='row',
        schema=['prop', 'value'],
    )
    chart = alt.Chart(contrib).mark_bar().encode(
        x=alt.X('value', scale=alt.Scale(domain=[0, 1])),
        y=alt.Y('prop', sort='-x'),
        tooltip=['prop', 'value'],
    ).properties(
        title=person,
    )
    charts.append(chart)
    
n_per_row = 3
chart = None
while charts:
    row_charts = charts[:3]
    row = reduce(lambda x, y: x | y, row_charts)
    if chart is None:
        chart = row
    else:
        chart &= row
    charts = charts[3:]
chart

alt.VConcatChart(...)